In [ ]:
import cv2
import numpy as np

# Set up video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Define region of interest (ROI) vertices
roi_vertices = [(0, 480), (320, 320), (320, 320), (640, 480)]

# Define color thresholds for lane detection
lower_yellow = np.array([18, 94, 140], dtype=np.uint8)
upper_yellow = np.array([48, 255, 255], dtype=np.uint8)
lower_white = np.array([0, 0, 200], dtype=np.uint8)
upper_white = np.array([255, 50, 255], dtype=np.uint8)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    
    # Apply color thresholding to isolate lane lines
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask = cv2.bitwise_or(mask_yellow, mask_white)
    
    # Apply a Gaussian blur to the masked image
    kernel_size = 5
    blur = cv2.GaussianBlur(mask, (kernel_size, kernel_size), 0)
    
    # Apply Canny edge detection to the blurred image
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur, low_threshold, high_threshold)
    
    # Mask the edges image using the ROI vertices
    mask_edges = np.zeros_like(edges)
    cv2.fillPoly(mask_edges, np.array([roi_vertices], dtype=np.int32), 255)
    masked_edges = cv2.bitwise_and(edges, mask_edges)
    
    # Apply Hough transform to detect lane lines
    rho = 1
    theta = np.pi/180
    threshold = 15
    min_line_length = 40
    max_line_gap = 20
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), 
                            min_line_length, max_line_gap)
    
    # Draw the detected lines on the original frame
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    
    # Overlay the lane lines on the original frame
    output = cv2.addWeighted(frame, 0.8, line_image, 1, 0)
    
    # Display the output frame
    cv2.imshow('Lane Detection', output)
    
    # Exit on key press
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()
